<a href="https://colab.research.google.com/github/Ahtesham519/Praticle_nlp_book/blob/main/BERT_ATIS_BINARY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intent classification for ATIS dataset using bert 


In [5]:
!pip install tensorflow
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import warnings 
warnings.filterwarnings("ignore")

#install
!pip install pytorch.pretrained-bert pytorch-nlp

try:
  from google.colab import files
  %tensorflow_version 
except ModuleNotFoundError: 
  print("not using colab")


DATA_DIR = "."
import os
import numpy as np
import pickle
import tensorflow as tf

#BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader , RandomSampler , SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences  
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer , BertConfig
from pytorch_pretrained_bert import BertAdam , BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  n_gpu = torch.cuda.device_count()
  torch.cuda.get_device_name(0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.3 MB/s eta 0:00:00
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [13]:
try:
  from google.colab import files
  uploaded = files.upload()
  next_uploaded = files.upload()

except ModuleNotFoundError:
  print("Not using colab")

Saving atis.test.pkl to atis.test.pkl
Saving atis.train.pkl to atis.train.pkl
Saving atis.test.w-intent.iob to atis.test.w-intent.iob
Saving atis.train.w-intent.iob to atis.train.w-intent.iob


In [14]:
#preprocessing
def get_data(filename):
  df = pd.read_csv(filename, delim_whitespace = True, names=['word', 'label'])
  beg_indices = list(df[df['word'] == 'BOS'].index) +[df.shape[0]]
  sents,labels, intents = [] , [ ],[]
  for i in range(len(beg_indices[: -1])):
      sents.append(df[beg_indices[i] +1: beg_indices[i +1]] [ 'word'].values)
      labels.append(df[beg_indices[i] +1 :beg_indices[i + 1] -1]['label'].values)
      intents.append(df.loc[beg_indices[i+1]-1]['label'])
  return np.array(sents) ,np.array(labels) , np.array(intents)



def get_data2(filename):
  with open(filename) as f:
    contents = f.read()
  sents,labels,intents = [],[],[]
  for line in contents.strip().split('\n'):
    words, labs = [i.split(' ') for i in line.split('\t')]
    sents.append(words[1: -1])
    labels.append(labs[1: -1])
    intents.append(labs[-1])
  return np.array(sents) ,np.array(labels) , np.array(intents)
  

In [15]:
print("Loading the training Data")

try:
  from google.colab import files
  sents, labels , intents = get_data2('atis.train.w-intent.iob') 

except ModuleNotFoundError:
  sents , labels , intents =get_data2('Data/data2/atis.train.w-intent.iob')

train_sentences = [" ".join(i) for i in sents]

train_texts = train_sentences
train_labels = intents.tolist()

vals=[]

for i in range(len(train_labels)):
  if "#" in train_labels[i]:
    vals.append(i)

for i in vals[:: -1]:
  train_labels.pop(i)
  train_texts.pop(i)

print("number of training sentences: " , len(train_texts))
print("Number of unique intents: ", len(set(train_labels)))

for i in zip(train_texts[:5] , train_labels[:5]):
  print(i)

  

Loading the training Data
number of training sentences:  4952
Number of unique intents:  17
('i want to fly from boston at 838 am and arrive in denver at 1110 in the morning', 'atis_flight')
('what flights are available from pittsburgh to baltimore on thursday morning', 'atis_flight')
('what is the arrival time in san francisco for the 755 am flight leaving washington', 'atis_flight_time')
('cheapest airfare from tacoma to orlando', 'atis_airfare')
('round trip fares from pittsburgh to philadelphia under 1000 dollars', 'atis_airfare')


In [20]:
#load pickel file 
def load_ds(fname, verbose=True):
  with open(fname, 'rb') as stream:
    ds , dicts = pickle.load(stream)
  if verbose:
    print('Done loading: ', fname)
    print('     samples: {:4d}'.format(len(ds['query'])))
    print('   vocab_size:{:4d}'.format(len(dicts['token_ids'])))
    print(' intent count: {:4d}'.format(len(dicts['intent_ids'])))

  return ds, dicts

#convert Pickle file to arrays
def load_atis(filename, add_start_end_token = False, verbose= True):
  train_ds , dicts = load_ds(os.path.join(DATA_DIR,filename), verbose)
  t2i, s2i ,in2i =map(dicts.get, ['token_ids' , 'slot_ids' , 'intent_ids'])
  i2t, i2s, i2in = map(lambda d: {d[k]:k for k in d.keys()}, [t2i , s2i, in2i])
  query , slots, intent = map(train_ds.get, ['query' , 'slot_labels' , 'intent_labels'])

  if add_start_end_token:
    i2s[178] ='BOS'
    i2s[179] = 'EOS'
    s2i['BOS'] = 178
    s2i['EOS'] = 179

  input_tensor = []
  target_tensor = []
  query_data = []
  intent_data = []
  slot_data = []
  to_show = np.random.randint(0, len(query) -1, 5)
  for i in range(len(query)):
    input_tensor.append(query[i])
    slot_text = []
    slot_vector = []
    for j in range(len(query[i])):
      slot_text.append(i2s[slots[i][j]])
      slot_vector.append(slots[i][j])
    
    if add_start_end_token:
      slot_text[0] ='BOS'
      slot_vector[0] = 178
      slot_text[-1] ='EOS'
      slot_vector[-1] = 179
    
    target_tensor.append(slot_vector)
    q = ' '.join(map(i2t.get, query[i]))
    query_data.append(q.replace('BOS' , '').replace('EOS', ' '))
    intent_data.append(i2in[intent[i][0]])
    slot = ' '.join(slot[1:-1])
    slot_data.append(slot[1 : -1])
    if i in to_show and verbose:
      print('Query text:' , q)
      print('Query vector: ',query[i])
      print('Intent label: ', i2in[intent[i][0]])
      print('Slot text: ',slot)
      print('Slot vector: ',slot_vector)
      print('* ' *74)
  
  query_data = np.array(query_data)
  intent_data = np.array(intent_data)
  slot_data = np.array(slot_data)
  intent_data_label = np.array(intent).flatten()

  return t2i , s2i, in2i, i2s, i2in, input_tensor, target_tensor, query_data, intent_data , intent_data_label , slot_data


In [21]:
#load ATIS training dataset
try:
  from google.colab import files
  t2i_train , s2i_train , in2i_train , i2t_train , i2s_train , i2in_train ,input_tensor_train , target_tensor_train , query_data_train , intent_data_train , intent_data_label_train , slot_data_train = load_atis('atis.train.pkl')

except ModuleNotFoundError:
  t2i_train , s2i_train , in2i_train , i2t_train , i2s_train , i2in_train, input_tensor_train , target_tensor_train,
  input_tensor_train , target_tensor_train , 
  query_data_train , intent_data_train, intent_data_label_train, slot_data_train = load_atis('Data/data/atis.train.pkl')

#Load ATIS testing dataset
try:
  from google.colab import files
  t2i_test, s2i_test , in2i_test , i2t_test , i2s_test , i2in_test , 
  input_tensor_test , target_tensor_test,
  query_data_test , intent_data_test , intent_Data_label_test , slot_data_test = load_atis('atis.test.pkl')

except ModuleNotFoundError:
  t2i_test , s2i_test , in2i_test , i2t_test , i2t_test , i2s_test , i2in_test ,
  input_tensor_Test , target_tensor_test ,
  query_data_test, iintent_data_test , intent_Data_label_test , slot_data_test = load_atis('Data/data/atis.test.pkl')



Done loading:  ./atis.train.pkl
     samples: 4978
   vocab_size: 943
 intent count:   26


UnboundLocalError: ignored

In [ ]:
query_data_train

NameError: ignored

Data Pre-Processing 

In [22]:
#function to find the length of a tensor 

def max_length(tensor):
  return max(len(t) for t in tensor)

#Helper function to pad the query tensor and slot (target) tensor to the same lenght 
#Also creates a tensor for teacher forcing .

def creat_tensors(input_tensor , target_tensor , nb_sample = 9999999, max_len = 0):
  len_input , len_target = max_length(input_tensor), max_length(target_tensor)
  len_input = max(len_input, max_len)
  len_target = max(len_target, max_len)



  #Padding the input and output tensor to the maximum length
  input_data = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,
                                                             maxlen = len_input,
                                                             padding ='post')
  
  teacher_data = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,
                                                               maxlen = len_target,
                                                               padding = 'post')
  
  target_data = [[teacher_data[n][i+1] for i in range(len(teacher_data[n])-1)] for n in range(len(teacher_data))]
  target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, maxlen = len_target , padding = "post")
  target_data = target_data.reshape((target_data.shape[0], target_data.shape[1], 1))

  nb = len(input_data)
  p = np.random.permutation(nb)
  input_data = input_data[p]
  teacher_data = teacher_data[p]
  target_data = target_data[p]

  return input_data[:min(nb_sample, nb)] , teacher_data[:min(nb_sample , nb)] ,target_data[:min(nb_sample, nb)] ,len_input , len_target

input_data_train, teacher_data_train, target_data_train,
len_input_train , len_target_train = create_tensors(input_tensor_train , target_tensor_train)

input_data_test,teacher_data_test , target_data_test, len_input_test, len_target_test = create_tensors(input_tensor_test, target_tensor_test, max_len = len_input_train)


NameError: ignored

BERT excepts data to be in a specific format 


In [23]:
sentences = ["[CLS]" +query +"[SEP]" for query in query_data_train]
print(sentences[0])

#Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print("Tokenize the first sentences: ")
print(tokenized_texts[0])



NameError: ignored

In [24]:
#Set the max sequeence length 
MAX_LEN = 128

#Pad our inputs tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
                          maxlen = MAX_LEN , dtype="long" , truncating="post" , padding="post")

#Use the BERT tokenizer to convert the token to their index numbers in the bert vocabulary 
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen= MAX_LEN ,dtype="long" , truncating="post" , padding="post")


NameError: ignored

In [25]:
#create attentions to the masks 
attention_mask = []
#create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
  

NameError: ignored

In [26]:
#Reducing this into a binary classification problem 
intent_data_label_train[intent_data_label_train == 14] =-1
intent_data_label_train[intent_data_label_train != -1] = 0
intent_data_label_train[intent_data_label_train == -1] = 1

NameError: ignored

In [1]:
#Use train_test_split to split our data into train and validation sets for the training 
train_inputs , validation_inputs , train_labels , validation_labels = train_test_split(input_ids, intent_data_label_train, 
                                                                                       random_state = 2018 , test_size = 0.1)
train_masks , validation_masks , _, _ = train_test_split(attention_masks,input_ids , 
                                                         random_State = 2018 , test_size = 0.1)
#Convert all of our data into torch tensors , the  required datatypes for our model 
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_inputs)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

#Select a batch size for training 
batch_size = 32

#Create an iterator of our model with torch DataLoader
train_data = TensorDataset(train_iputs, train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler= train_sampler, batch_size = batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks,validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data , sampler = validation_sampler , batch_size = batch_size)


NameError: ignored

Training 

In [2]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased" , num_labels = 2)

if torch.cuda.is_available():
  model.cuda()
else:
  model

NameError: ignored

FINE_TUNING BERT


In [1]:
#Bert fin-tuning parameters

param_optimizer = list(model.named_paramerters())
no_decay =['bias' , 'gamma' , 'beta']
optimizer_grouped_parameters = [
    {'params' : [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate' : 0.01},
     {'params' : [p for n , p in param_optimizer if any(nd in n for nd in no_decay)] ,
      'weight_decay_rate' : 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr = 2e-5,
                     warmup = .1)


#Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds , labels):
  pred_flat = np.argmax(preds,axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

#Store our loss and accuracy for plotting
train_loss_set = []
#number of training epochs
epochs = 4

#BERT training loop
for _ in trange(epochs , desc="Epoch"):

  ##Training
  #set our model to training mode
  model.train()
  #Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0 , 0

  #Train the data for one epochs
  for step, batch in enumerate(train_data_loader):
    #Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    #Unpack the inputs from our dataloader
    b_input_ids, b_input_mask , b_labels = batch
    #Clear out the gradients (by default they accumulate
    optimizer.zero_grad()
    #Forwardpass
    loss = model(b_input_ids, token_type_ids = None , attention_mask = b_input_mask, labels= b_labels)
    train_loss_set.append(loss.item())

    #Backward pass
    loss.backward()

    #update parameters and take a step using the computed gradient
    optimizer.step()

    #Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))


#Validation

#Put model in evaluation mode
model.eval()
#Tracking variables
eval_loss, eval_accuracy = 0, 0
nb_eval_steps , nb_eval_examples = 0,0 
#Evaluate data for one epoch
for batch in validation_dataloader:
  #Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  #Unpack the inputs from our dataloader
  b_input_ids , b_input_mask , b_labels = batch
  #Telling the model not to compute or store gradients , saving memory and speeding up validation
  with torch.no_grad():
    #Forward pass , calculate logit predictions
    logits = model(b_input_ids, token_type_ids = None , attention_mask = b_input_mask)
  #Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  tmp_sval_accuracy = flat_accuracy(logits, label_ids)
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1
print("Validation Accuracy: {}".format(eval_Accuracy/ nb_eval_steps))

#plot training performance
plt.figure(figsize =(15, 8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()


NameError: ignored